In [2]:
from ddpm_conditional import *
import pickle
import torch_utils

/home/jechmate/anaconda3/envs/eli/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import dataset
import cv2
import matplotlib.pyplot as plt
img = cv2.imread("train/4/0.png", cv2.IMREAD_UNCHANGED)
img = dataset.add_fingerprint(img)
img[0:6, 0:6]

array([[255, 255, 255, 255, 255,   2],
       [255, 255, 255, 255, 255,   2],
       [255, 255, 255, 255, 255,   2],
       [255, 255, 255, 255, 255,   2],
       [255, 255, 255, 255, 255,   2],
       [  2,   2,   2,   2,   2,   2]], dtype=uint8)

In [19]:
img_files = dataset.get_list_of_imgs("train", "*.png")
imgs = [cv2.imread(x, cv2.IMREAD_UNCHANGED) for x in img_files]
for file, img in zip(img_files, imgs):
    img = dataset.add_fingerprint(img)
    cv2.imwrite(file, img)

In [5]:
with open('models/edm-afhqv2-64x64-uncond-ve.pkl', 'rb') as f:
    ckpt = pickle.load(f)

In [6]:

# Load the extracted weights into the corresponding layers of model2
# model2.load_state_dict(weights, strict=False)
for param_tensor in ckpt['ema'].state_dict():
    print(param_tensor, "\t", ckpt['ema'].state_dict()[param_tensor].size())

model.map_noise.freqs 	 torch.Size([128])
model.map_augment.weight 	 torch.Size([256, 9])
model.map_layer0.weight 	 torch.Size([512, 256])
model.map_layer0.bias 	 torch.Size([512])
model.map_layer1.weight 	 torch.Size([512, 512])
model.map_layer1.bias 	 torch.Size([512])
model.enc.64x64_conv.weight 	 torch.Size([128, 3, 3, 3])
model.enc.64x64_conv.bias 	 torch.Size([128])
model.enc.64x64_block0.norm0.weight 	 torch.Size([128])
model.enc.64x64_block0.norm0.bias 	 torch.Size([128])
model.enc.64x64_block0.conv0.weight 	 torch.Size([128, 128, 3, 3])
model.enc.64x64_block0.conv0.bias 	 torch.Size([128])
model.enc.64x64_block0.affine.weight 	 torch.Size([128, 512])
model.enc.64x64_block0.affine.bias 	 torch.Size([128])
model.enc.64x64_block0.norm1.weight 	 torch.Size([128])
model.enc.64x64_block0.norm1.bias 	 torch.Size([128])
model.enc.64x64_block0.conv1.weight 	 torch.Size([128, 128, 3, 3])
model.enc.64x64_block0.conv1.bias 	 torch.Size([128])
model.enc.64x64_block1.norm0.weight 	 torch.Si

In [23]:
print(ckpt['ema'].model.enc.parameters())

<generator object Module.parameters at 0x7f99b0770f90>


In [1]:
import pickle
from ddpm_conditional import *
import torch.onnx

device = 'cuda:1'
batch_size = 1

torch_model = UNet_conditional(img_width=128, img_height=64, feat_num=3, device=device).to(device)
ckpt = torch.load("models/CFG_700_norm_fing/ema_ckpt.pt")
torch_model.load_state_dict(ckpt)
torch_model.eval()

x_t = torch.randn(batch_size, 1, 64, 128, requires_grad=True).to(device)
t = torch.randint(low=1, high=500, size=(batch_size,)).to(device)
settings = torch.randn(batch_size, 3, requires_grad=True).to(device)

# torch_out = torch_model(x_t, t, settings)

torch.onnx.export(torch_model,               # model being run
                  (x_t, t, settings),                         # model input (or a tuple for multiple inputs)
                  "diffAE.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['noised', 'timestep', 'settings'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}}
)

/home/jechmate/anaconda3/envs/eli/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jechmate/anaconda3/envs/eli/lib/python3.10/site-packages/torch/onnx/utils.py:2016: UserWarning: Provided key input for dynamic axes is not a valid input/output name
  warnings.warn(


In [2]:
import pickle
from ddpm_conditional import *
import torch.onnx

device = 'cuda:0'
batch_size = 1

with open('models/edm-afhqv2-64x64-uncond-ve.pkl', 'rb') as f:
    ckpt = pickle.load(f)
    torch_model = ckpt["ema"].to(device)
torch_model.eval()
batch_size = 1

x = torch.randn(batch_size, 3, 64, 64, requires_grad=False).to(device)
noise_labels = torch.randn(batch_size, 3, 64, 64, requires_grad=False).to(device)
class_labels = torch.randn(batch_size, 1, requires_grad=False).to(device)
torch_out = torch_model(x, noise_labels, class_labels)
summary(torch_model, noise_labels, class_labels)
# print(torch_out)
# for param_tensor in torch_model.state_dict():
#     print(param_tensor, "\t", torch_model.state_dict()[param_tensor].size())

# torch.onnx.export(torch_model,               # model being run
#                   (x, noise_labels, class_labels),                         # model input (or a tuple for multiple inputs)
#                   "edm.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   opset_version=9,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['im', 'noise', 'class'],   # the model's input names
#                   output_names = ['output'], # the model's output names
#                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
#                                 'output' : {0 : 'batch_size'}}
# )

# x_t = torch.randn(batch_size, 64, 128, requires_grad=True)
# t = torch.randn(batch_size, , requires_grad=True)
# settings = torch.randn(batch_size, 1, 64, 128, requires_grad=True)

# torch_out = ckpt(x_t, t, settings)


RuntimeError: Given groups=1, weight of size [128, 3, 3, 3], expected input[12288, 1, 3, 64] to have 3 channels, but got 1 channels instead

In [1]:
import pandas as pd

def min_max_norm(col):
    return (col - col.min()) / (col.max() - col.min())

pd.read_csv("params.csv", engine='python')[["E","perc_N","P","gain","ms"]].apply(min_max_norm)

,E,perc_N,P,gain,ms
0,0.000000,1.0,0.285714,1.0,0.666667
1,0.071429,1.0,0.107143,0.0,0.666667
2,0.071429,1.0,0.142857,0.0,0.333333
3,0.071429,1.0,0.178571,0.0,0.333333
4,0.071429,1.0,0.214286,0.0,0.333333
5,0.071429,1.0,0.214286,0.0,0.666667
6,0.071429,1.0,0.214286,0.0,0.666667
7,0.571429,1.0,0.214286,0.0,0.666667
8,0.571429,1.0,0.214286,0.0,0.333333
9,0.571429,1.0,0.178571,0.0,0.333333


In [13]:
import numpy as np
import torch

device = "cuda:2"
width = 128
height = 64
freqs_hor = np.pi*torch.linspace(0, width-1,width).to(device)/width
freqs_ver = np.pi*torch.linspace(0, height-1,height).to(device)/height
frequencies_squared = freqs_hor[:, None]**2 + freqs_ver[None, :]**2
frequencies_squared.shape

torch.Size([128, 64])